# Загрузка библиотек

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import time
import albumentations as A
from PIL import Image
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Создание аугментированных фото с "антиклассами"

In [3]:
def lambda_image(x): return Image.fromarray(x)

def channels_swap(img):
    # chanels swap
    im = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2RGB)
    return lambda_image(im)

def blur(img):
    # image blur
    im = cv2.medianBlur(np.asarray(img), 3)
    return lambda_image(im)

def contrast(img):
    # contrast change
    trans_contrast = (
        A.RandomBrightnessContrast(
            always_apply=True,
            contrast_limit=(-.6, -.6),
            p=1
        )
    )
    im = trans_contrast(image=img)['image']
    return lambda_image(im)

def spatters(img):
    # spatters addition
    trans_spat = (
        A.Spatter(
            always_apply=True, mean=-0.79,
            std=13.16, p=1
        )
    )
    im = trans_spat(image=img)['image']
    return lambda_image(im)


def clache(img):
    # clache addition
    trans_clache = (
        A.CLAHE(
            always_apply=True,
            p=1
        )
    )
    im = trans_clache(image=img)['image']
    return lambda_image(im)


def emboss(img):
    # emboss addition
    trans_emboss = (
        A.Emboss(
            always_apply=True,
            p=1
        )
    )
    im = trans_emboss(image=img)['image']
    return lambda_image(im)

In [4]:
if not os.path.exists('../data/sorted_data_merged/damaged_augm'):
    os.makedirs('../data/sorted_data_merged/damaged_augm')

In [27]:
trans_contrast = A.RandomBrightnessContrast(
        always_apply=True,
        contrast_limit=(0.6, 0.77),
        brightness_limit=(0, 0),
        p=1
    )

trans_spat = A.Spatter(
        always_apply=True, mean=-0.79,
        std=13.16, p=1
    )

trans_clache = A.CLAHE(
        always_apply=True,
        p=1
    )

trans_emboss = A.Emboss(
    always_apply=True,
    p=1
)

trans_sharp = A.Sharpen(
    always_apply=True,
    p=1
)

trans_random_gamma = A.RandomGamma(
    always_apply=True,
    p=1
)

trans_hue_sat = A.HueSaturationValue(
    always_apply=True,
    hue_shift_limit=(-20, 20),
    sat_shift_limit=(-30, 30),
    val_shift_limit=(-20, 20),
    p=1
)

trans_color_jitter = A.ColorJitter(
    always_apply=True,
    brightness=0.2,
    contrast=0.2,
    saturation=0.2,
    hue=0.2,
    p=1
)

trans_random_crop = A.RandomCrop(
    always_apply=True,
    height=800,
    width=800,
    p=1
)

trans_random_rotate = A.RandomRotate90(
    p=1,
    always_apply=True
)

trans_coarse_dropout= A.CoarseDropout(
    p=1, always_apply=True
)

trans_random_fog = A.RandomFog(
    fog_coef_lower=0.1,
    fog_coef_upper=0.15,
    alpha_coef=0.15,
    p=1, always_apply=True
)

trans_unsharp = A.UnsharpMask(
    always_apply=True, p=1
)

trans_to_gray = A.ToGray(
    always_apply=True, p=1
)

trans_random_combs = A.Compose([
    trans_hue_sat,
    trans_color_jitter,
    trans_sharp,
    trans_to_gray
])

trans_random_combs2 = A.Compose([
    trans_hue_sat,
    trans_unsharp, 
])

trans_random_combs3 = A.Compose([
    trans_random_gamma,
    trans_hue_sat,
    trans_color_jitter,
    trans_clache
])

trans_contrast_gray_scale = A.Compose([
    trans_to_gray,
    trans_contrast
])

trans_final_transfrom = A.Compose([
    trans_contrast, 
    trans_random_rotate,
    trans_coarse_dropout
])

In [28]:
data_folder = '../data/sorted_data_merged/'
for file in tqdm(os.listdir(data_folder+'damaged')):
    image = np.array(Image.open(data_folder+'damaged/'+file))

    # start_time = time.time()
    image_contr = trans_contrast(image=image)['image']
    image_contr = trans_final_transfrom(image=image_contr)['image']
    im = Image.fromarray(image_contr)
    im.save(f"{data_folder+'damaged_augm'}/{file}_contr.jpeg")
    # print('Contr:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    # image_spat = trans_spat(image=image)['image']
    # image_spat = trans_final_transfrom(image=image_spat)['image']
    # im = Image.fromarray(image_spat)
    # im.save(f"{data_folder+'damaged_augm'}/{file}_spat.jpeg")
    # print('Spat:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_random_combs2 = trans_random_combs2(image=image)['image']
    image_random_combs2 = trans_final_transfrom(image=image_random_combs2)['image']
    im = Image.fromarray(image_random_combs2)
    im.save(f"{data_folder+'damaged_augm'}/{file}_random_combs2.jpeg")
    # print('Random Combs2:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_clache = trans_clache(image=image)['image']
    image_clache = trans_final_transfrom(image=image_clache)['image']
    im = Image.fromarray(image_clache)
    im.save(f"{data_folder+'damaged_augm'}/{file}_clache.jpeg")
    # print('Clache:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_emboss = trans_emboss(image=image)['image']
    image_emboss = trans_final_transfrom(image=image_emboss)['image']
    im = Image.fromarray(image_emboss)
    im.save(f"{data_folder+'damaged_augm'}/{file}_emboss.jpeg")
    # print('Emboss:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_sharp = trans_sharp(image=image)['image']
    image_sharp = trans_final_transfrom(image=image_sharp)['image']
    im = Image.fromarray(image_sharp)
    im.save(f"{data_folder+'damaged_augm'}/{file}_sharp.jpeg")
    # print('Sharp:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_random_combs3 = trans_random_combs3(image=image)['image']
    image_random_combs3 = trans_final_transfrom(image=image_random_combs3)['image']
    im = Image.fromarray(image_random_combs3)
    im.save(f"{data_folder+'damaged_augm'}/{file}_random_combs3.jpeg")
    # print('Random combs 3:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_hue_sat = trans_hue_sat(image=image)['image']
    image_hue_sat = trans_final_transfrom(image=image_hue_sat)['image']
    im = Image.fromarray(image_hue_sat)
    im.save(f"{data_folder+'damaged_augm'}/{file}_hue_sat.jpeg")
    # print('Hue Sat:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_color_jitter = trans_color_jitter(image=image)['image']
    image_color_jitter = trans_final_transfrom(image=image_color_jitter)['image']
    im = Image.fromarray(image_color_jitter)
    im.save(f"{data_folder+'damaged_augm'}/{file}_color_jitter.jpeg")
    # print('Collor Jit:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_random_combs = trans_random_combs(image=image)['image']
    image_random_combs = trans_final_transfrom(image=image_random_combs)['image']
    im = Image.fromarray(image_random_combs)
    im.save(f"{data_folder+'damaged_augm'}/{file}_random_combs.jpeg")
    # print('Random Combs:\t'+ "--- %s seconds ---" % (time.time() - start_time))

    # start_time = time.time()
    image_gray_contr = trans_contrast_gray_scale(image=image)['image']
    image_gray_contr = trans_final_transfrom(image=image_gray_contr)['image']
    im = Image.fromarray(image_gray_contr)
    im.save(f"{data_folder+'damaged_augm'}/{file}_gray_contr.jpeg")
    # print('Gray Contr:\t'+ "--- %s seconds ---" % (time.time() - start_time))
    # print()

100%|██████████| 1072/1072 [24:54<00:00,  1.39s/it]
